In [13]:
import pandas as pd
import numpy as np
import pickle
from string import punctuation

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
/content/drive/MyDrive/sentiment-analysis-hse-fcl


In [4]:
%cd /content/drive/MyDrive/russian_sentiment_tweet_automated_labeled

/content/drive/MyDrive/russian_sentiment_tweet_automated_labeled


In [5]:
data_pos = pd.read_csv('positive.csv', delimiter=';', header=None)
data_neg = pd.read_csv('negative.csv', delimiter=';', header=None)

In [6]:
data_pos = data_pos[3]
data_neg = data_neg[3]
data_pos = pd.DataFrame(data_pos)
data_neg = pd.DataFrame(data_neg)
data_pos = data_pos.rename(index=int, columns={3: "text"})
data_neg = data_neg.rename(index=int, columns={3: "text"})
data_pos['labels'] = 1
data_neg['labels'] = 0

In [7]:
len(data_pos), len(data_neg)

(114911, 111923)

In [27]:
import re
import nltk
from nltk import (sent_tokenize as splitter,wordpunct_tokenize as tokenizer)
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

def lowercase(text):
    return text.lower()

def remove_url(text):
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'www\S+', '', text)
    return text

def remove_punctuation(text):
    return ''.join([word for word in text if word not in punctuation])

# Remove irrelevant Characters (Numbers and Punctuation)
def remove_irr_char_func(text):
    return re.sub(r'[^А-яЁё]', ' ', text)

# Remove stopwords
def clean_stop_words(text, stopwords):
    text = [word for word in text.split() if word not in stopwords]
    return " ".join(text)

def tokenize(text):
    return [tokenizer(sentence) for sentence in splitter(text)]

def flatten(nested_list):
    return [item for sublist in nested_list for item in sublist]

def tokenize_flatten_df(row, field):
    return flatten(tokenize(row[field]))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [30]:
from nltk.corpus import stopwords as nltk_stopwords

# загрузим список стоп-слов
stopwords = set(nltk_stopwords.words('russian'))
np.array(stopwords)

array({'там', 'ней', 'во', 'вас', 'ну', 'здесь', 'если', 'более', 'еще', 'много', 'мне', 'никогда', 'ему', 'три', 'от', 'ей', 'их', 'на', 'он', 'о', 'всего', 'к', 'но', 'том', 'сам', 'этом', 'без', 'раз', 'почти', 'мы', 'мой', 'есть', 'ее', 'эти', 'теперь', 'нас', 'чего', 'про', 'где', 'тоже', 'конечно', 'такой', 'будто', 'была', 'по', 'нельзя', 'были', 'вдруг', 'вам', 'когда', 'под', 'что', 'как', 'тут', 'свою', 'уже', 'один', 'ним', 'нее', 'или', 'через', 'у', 'быть', 'уж', 'я', 'совсем', 'моя', 'всю', 'опять', 'кто', 'между', 'а', 'для', 'зачем', 'после', 'было', 'разве', 'куда', 'над', 'него', 'этой', 'они', 'за', 'лучше', 'бы', 'так', 'с', 'чтоб', 'ты', 'тогда', 'всех', 'иногда', 'себя', 'этот', 'при', 'наконец', 'того', 'не', 'ведь', 'чтобы', 'два', 'об', 'хоть', 'нет', 'им', 'со', 'надо', 'тот', 'этого', 'да', 'сейчас', 'меня', 'чем', 'какая', 'ничего', 'в', 'был', 'потому', 'и', 'из', 'вы', 'впрочем', 'до', 'ж', 'то', 'перед', 'какой', 'ни', 'них', 'потом', 'чуть', 'хорошо', 'с

In [28]:
data = pd.concat([data_pos, data_neg], ignore_index=True)

In [31]:
data['text_without_urls'] = data['text'].apply(lambda x: remove_url(lowercase(x)))

data['text-clear'] = data['text_without_urls'].apply(lambda x: remove_irr_char_func(remove_punctuation(clean_stop_words(x, stopwords))))

data['tokenized-text'] = \
    data.apply(
        lambda row:
            tokenize_flatten_df (row, 'text-clear'),
        axis=1)

In [32]:
data.head()

,text,labels,text_without_urls,text-clear,tokenized-text
0,"@first_timee хоть я и школота, но поверь, у на...",1,"@first_timee хоть я и школота, но поверь, у на...",школота поверь самое общество про...,"[школота, поверь, самое, общество, профилирующ..."
1,"Да, все-таки он немного похож на него. Но мой ...",1,"да, все-таки он немного похож на него. но мой ...",да всетаки немного похож него мальчик равно лу...,"[да, всетаки, немного, похож, него, мальчик, р..."
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,rt @katiacheh: ну ты идиотка) я испугалась за ...,идиотка испугалась тебя,"[идиотка, испугалась, тебя]"
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,"rt @digger2912: ""кто то в углу сидит и погибае...",кто углу сидит погибает голода е...,"[кто, углу, сидит, погибает, голода, ещё, порц..."
4,@irina_dyshkant Вот что значит страшилка :D\nН...,1,@irina_dyshkant вот что значит страшилка :d\nн...,значит страшилка блинпосмотрев...,"[значит, страшилка, блинпосмотрев, частиу, соз..."


In [33]:
data = data.drop(['text', 'text_without_urls', 'text-clear'], axis=1)
data.columns = ['label', 'text']

In [35]:
data = data[data['text'].map(lambda d: len(d)) > 0]

In [36]:
data['text'] = [[i for i in d if i!=''] for d in data['text']]
data['text'] = [" ".join(d) for d in data['text']]
data['text'] = [d.strip() for d in data['text']]

<ipython-input-36-9c89f268898f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'] = [[i for i in d if i!=''] for d in data['text']]
<ipython-input-36-9c89f268898f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'] = [" ".join(d) for d in data['text']]
<ipython-input-36-9c89f268898f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas

In [37]:
empty_rows = []
for i , t in enumerate(data['text'].values):
    if t == '':
        empty_rows.append(i)

In [38]:
data.drop(data.index[empty_rows], inplace=True)

<ipython-input-38-5e175ad819b1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.drop(data.index[empty_rows], inplace=True)


In [39]:
data

,label,text
0,1,школота поверь самое общество профилирующий пр...
1,1,да всетаки немного похож него мальчик равно лучше
2,1,идиотка испугалась тебя
3,1,кто углу сидит погибает голода ещё порции взял...
4,1,значит страшилка блинпосмотрев частиу создастс...
...,...,...
226829,0,каждый хочет исправлять
226830,0,скучаю вправляет мозги равно скучаю
226831,0,школу говно это идти надо
226832,0,тауриэль грусти обнял


In [40]:
data['text_tokenized'] = \
    data.apply(
        lambda row:
            tokenize_flatten_df (row, 'text'),axis=1)

<ipython-input-40-c0533255c35e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text_tokenized'] = \


In [41]:
data = data.drop(['text'], axis=1)
data.columns = ['label', 'text']

In [42]:
data

,label,text
0,1,"[школота, поверь, самое, общество, профилирующ..."
1,1,"[да, всетаки, немного, похож, него, мальчик, р..."
2,1,"[идиотка, испугалась, тебя]"
3,1,"[кто, углу, сидит, погибает, голода, ещё, порц..."
4,1,"[значит, страшилка, блинпосмотрев, частиу, соз..."
...,...,...
226829,0,"[каждый, хочет, исправлять]"
226830,0,"[скучаю, вправляет, мозги, равно, скучаю]"
226831,0,"[школу, говно, это, идти, надо]"
226832,0,"[тауриэль, грусти, обнял]"


In [43]:
with open('twitter_prep_data_brackets.pickle', 'wb') as f:
        pickle.dump(data, f)